In [54]:
import json
import os
import numpy as np
import datetime
from skimage import io
from osgeo import gdal, osr
import sys

In [58]:
class Dataset(object):
    def __init__(self):
        now = datetime.datetime.now()
        self.info_info = [{'year':now.year,'version':'','description':'','contributor':'','url':'','date_created':str(now)}]
        # Background is always the first class
        #list of dictionaries
        self.license_info = [{'id': 0, 'name': 'GEOJsonToCOCO', 'url': ''}]
        #self.categories_info = [{'id': 0, 'name': 'BG', 'supercategory': ''}]
        self.categories_info = []
        #list of dictionaries
        self.images_info = []
        self.annotations_info = []
        
    def load_geojson_file(self, filePath, imagePath):
        json_file = open(filePath)
        geo_json = json.load(json_file)
        json_file.close()
        
        
        imageFilename = os.path.split(imagePath)[1]
        #split out the digits at the end of the filename
        imageID = int(imageFilename.split('.')[0].split('_')[-1][3:])
        
        src_raster = gdal.Open(imagePath)
        geom_transform = src_raster.GetGeoTransform()
        longCorner = geom_transform[0]
        latCorner = geom_transform[3]
        pixelWidth = geom_transform[1]
        pixelHeight = geom_transform[5]
        
        try:
            #Check to ensure there are annotations in the file
            if geo_json['features'] != []:
                #Add polygons
                for feature in geo_json['features']:
                    #Check to make sure that each feature is more than two coordinates:
                    #if len(feature['geometry']['coordinates'][0]) > 2:

                    if feature['geometry']['type'] == 'Polygon':
                        for coordinatesSet in feature['geometry']['coordinates']:
                            pixelCoordinates, xPixels, yPixels = self.convert_coordinatesSet(coordinatesSet, longCorner, latCorner, pixelWidth, pixelHeight)
                            polygonArea = self.polygon_area(np.asarray(xPixels), np.asarray(yPixels))
                            #For all but Shanghai
                            #objectID = feature['properties']['OBJECTID_1']
                            #For Shanghai
                            objectID = 0
                            category_id = 1
                            self.add_annotation(objectID, imageID, category_id, pixelCoordinates, polygonArea)

                    if feature['geometry']['type'] == 'MultiPolygon':
                         for polygon in feature['geometry']['coordinates']:
                            for coordinatesSet in polygon:
                                pixelCoordinates, xPixels, yPixels = self.convert_coordinatesSet(coordinatesSet, longCorner, latCorner, pixelWidth, pixelHeight)
                                polygonArea = self.polygon_area(np.asarray(xPixels), np.asarray(yPixels))
                                #For all but Shanghai
                                #objectID = feature['properties']['OBJECTID_1']
                                #For Shanghai
                                objectID = 0
                                category_id = 1
                                self.add_annotation(objectID, imageID, category_id, pixelCoordinates, polygonArea)                        
        except:
                print("Error: ", filePath)
            #Check to make an annotation exists for this image (met the criteria above) before adding the image to the list
            

        if next((item for item in self.annotations_info if item["image_id"] == imageID), False):
            try:
                image = skimage.io.imread(imagePath)
                imageWidth = image.shape[0]
                imageHeight = image.shape[1]
            except:
                print("Unexpected error:", sys.exc_info()[0])
                pass
            #Add image to the image_info list
            self.add_image(imageFilename, imageID, imageWidth, imageHeight) 
            

            #for feature in geo_json['features']:
            #    cat_id = feature['properties']['OBJECTID']
            #    cat_name = feature['geometry']['type']
            #    if cat_id < 1:
            #        cat_id = cat_id + 1
            #    self.add_class(cat_id, cat_name)
        
       
    def polygon_area(self, x, y):
        correction = x[-1] * y[0] - y[-1]* x[0]
        main_area = np.dot(x[:-1], y[1:]) - np.dot(y[:-1], x[1:])
        return 0.5*np.abs(main_area + correction)
    
    def convert_coordinatesSet(self, coordinatesSet, longCorner, latCorner, pixelWidth, pixelHeight):
        pixelCoordinates = []
        xPixels = []
        yPixels = []
        for coordinates in coordinatesSet:
            xcoord = coordinates[0]
            ycoord = coordinates[1]
            xPixel = (xcoord - longCorner) / pixelWidth
            yPixel = (ycoord - latCorner) / pixelHeight
            pixelCoordinates.append(xPixel)
            pixelCoordinates.append(yPixel)
            xPixels.append(xPixel)
            yPixels.append(yPixel)
        return pixelCoordinates, xPixels, yPixels
    
    def add_annotation(self, objectID, imageID, category_id, pixelCoordinates, polygonArea):
        self.annotations_info.append({'id' : objectID, 
                                       'image_id' : imageID, 
                                       'category_id' : category_id, 
                                       'segmentation' : [pixelCoordinates], 
                                       'area' : polygonArea, 
                                       'bbox' : [], 
                                       'iscrowd' : 0})
                    
    def add_image(self, imageFilename, imageID, imageWidth, imageHeight):
        self.images_info.append({'id':imageID,
                                 'width':imageWidth,
                                 'height':imageHeight,
                                 'file_name': imageFilename,
                                 'license':'',
                                 'flicker_url':'',
                                 'coco_url':'',
                                 'date_captured':''})
        
    def load_data(self, annotation_dir, images_dir):
        #Only here to speed up adding categories (we're only dealing with one category for buildings) 
        self.add_category(1, 'Feature')
        
        for jsonFile in os.listdir(annotation_dir):
            if jsonFile.split('.')[-1] == 'geojson':
                jsonFilename = jsonFile.split('_', 1)[1:][0].split('.',1)[0]
                #Pull the folder name for the image_dir (PAN, MIL, etc)
                imageType = image_dir.split('/')[-2]
                #Assemble image name that corresponds with the annotation file
                image = imageType + '_' + jsonFilename + '.tif'
                imagePath = os.path.join(image_dir, image)
                    
                filePath = os.path.join(annotation_dir, jsonFile)
                    
                #Execute for every geojson file in the folder
                self.load_geojson_file(filePath, imagePath)
                
    def add_category(self, cat_id, cat_name):
        if next((item for item in self.categories_info if item["id"] == cat_id), False):
            # cat_id combination already available, skip
            return
        self.categories_info.append({
            "id": cat_id,
            "name": cat_name,
            "supercategory": ''
        })        
    
    def write_json(self, outputName):
        json_dict = {'license': self.license_info, 'info': self.info_info, 'categories': self.categories_info,
                     'images': self.images_info, 'annotations': self.annotations_info}
        with open(outputName, 'w', encoding='utf-8') as f:
            json.dump(json_dict, f, ensure_ascii=False, indent=4)
        

In [59]:
dataset = Dataset()
annotation_dir = '/data/spacenet/bldg/AOI_4_Shanghai_Train/geojson/buildings/'
image_dir = '/data/spacenet/bldg/AOI_4_Shanghai_Train/PAN/'
dataset.load_data(annotation_dir, image_dir)
dataset.write_json('ShanghaiTrainBLDG.json')

In [13]:
dataset

In [55]:
image = skimage.io.imread('/data/spacenet/bldg/AOI_4_Shanghai_Train/PAN/PAN_AOI_4_Shanghai_img1001.tif')
imageWidth = image.shape[0]
imageHeight = image.shape[1]